In [2]:
import xarray as xr

In [7]:
# URL du dataset WeatherBench 2 sur Google Cloud
url = "gs://weatherbench2/datasets/era5/1959-2023_01_10-wb13-6h-1440x721_with_derived_variables.zarr"

# Ouverture lazy (ne charge rien encore en mémoire)
ds = xr.open_zarr(url, consolidated=True)

In [8]:
years = ["2000"]
ds = ds.sel(time=ds.time.dt.year.isin([int(y) for y in years]))

In [9]:
region=(-12.5, 42.5, 35,72)

lon_min, lon_max, lat_min, lat_max = region

# Sélection de la date et conversion en mm
# precip = ds["total_precipitation_6hr"].sel(
#     time=date, method="nearest", tolerance="3H"
# ) * 1000
precip = ds["total_precipitation_6hr"]* 1000

# Recentrage des longitudes sur [-180, 180]
precip = precip.assign_coords(
    longitude=(((precip.longitude + 180) % 360) - 180)
).sortby("longitude")

precip_region = precip.sel(
    longitude=slice(lon_min, lon_max),
    latitude=slice(lat_max, lat_min)
)

print(precip_region)

<xarray.DataArray 'total_precipitation_6hr' (time: 1464, latitude: 149,
                                             longitude: 221)> Size: 193MB
dask.array<getitem, shape=(1464, 149, 221), dtype=float32, chunksize=(1, 149, 221), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 12kB 2000-01-01 ... 2000-12-31T18:00:00
  * latitude   (latitude) float32 596B 72.0 71.75 71.5 71.25 ... 35.5 35.25 35.0
  * longitude  (longitude) float32 884B -12.5 -12.25 -12.0 ... 42.0 42.25 42.5


In [ ]:
number_rain_pixels=0
number_times=0
total_times=len(precip_region.time)
for time in precip_region.time:
    number_times+=1
    print(f"Processing time {number_times}/{total_times}")
    precip_time=precip_region.sel(time=time)
    number_rain_pixels+=(precip_time.values>=1).sum()
    print(f"Proportion of rain pixels so far: {number_rain_pixels/(number_times*32560)}")

Processing time 1/1464
Proportion of rain pixels so far: 0.1471130221130221
Processing time 2/1464
Proportion of rain pixels so far: 0.14691339066339065
Processing time 3/1464
Proportion of rain pixels so far: 0.15175061425061426
Processing time 4/1464
Proportion of rain pixels so far: 0.15491400491400492
Processing time 5/1464
Proportion of rain pixels so far: 0.15555896805896807
Processing time 6/1464
Proportion of rain pixels so far: 0.15513923013923014
Processing time 7/1464
Proportion of rain pixels so far: 0.15053527553527554
Processing time 8/1464
Proportion of rain pixels so far: 0.14845285626535626
Processing time 9/1464
Proportion of rain pixels so far: 0.15155268905268907
Processing time 10/1464
Proportion of rain pixels so far: 0.15601965601965603
Processing time 11/1464
Proportion of rain pixels so far: 0.15964094259548806
Processing time 12/1464
Proportion of rain pixels so far: 0.16157094594594595
Processing time 13/1464
Proportion of rain pixels so far: 0.16171328671328